# Libraries and dataset import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

from tensorflow import keras
from keras import Sequential, layers
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
df = pd.read_csv('ML-CUP22-TR.csv', header=None, skiprows=7, index_col=0) #, skipinitialspace=True)
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
X = df.iloc[:, 0:9].values
X

In [ ]:
y = df.iloc[:, 9:11].values
y

# Partitioning

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Data Exploration

In [ ]:
sns.heatmap(df.corr(), annot=True, vmin=-1, vmax=1)
plt.show()

In [ ]:
df.describe()

In [ ]:
def feature_selection(X_train, y_train, X_test, scale=True, score_func=r_regression):
    if scale:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
    kbest = SelectKBest(score_func = r_regression, k = "all")
    X_train = kbest.fit_transform(X_train,y_train)
    X_test = kbest.transform(X_test)

    res = pd.DataFrame(columns=['Feature', 'score'])
    for i, (f,s) in enumerate(zip(df.drop(columns=[10,11]).columns, kbest.scores_)):
        res.loc[i, 'Feature'] = f
        res.loc[i, 'score'] = s
    res = res.sort_values(by='score', ascending=False)
    return res

In [ ]:
a = feature_selection(X_train, y_train1, X_test, score_func=r_regression, scale=False)
b = feature_selection(X_train, y_train2, X_test, score_func=r_regression, scale=False)
pd.concat([a.rename(columns={"Feature": "Feature", "score": "score_y1"}), b.rename(columns={"Feature": "a", "score": "score_y2"})], axis=1)\
.drop(columns=['a'])\
.sort_values(by='score_y1',ascending=False)\
.reset_index(drop=True)


# Support Functions

In [ ]:
def mean_euclidean_error(y_true, y_pred):
    return K.mean(K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1)))

In [ ]:
def mean_euclidean_error_skit_friendly(T, O):
    sum = 0
    for t, o in zip(T, O):
        sum += np.linalg.norm(t - o) / T.shape[0]
    return sum

In [ ]:
def create_model_RMSprop(num_neurons, l_rates, decays, momentum):
        num_layers = len(num_neurons)
        model = Sequential()
        model.add(Dense(num_neurons[0], input_shape=(9,), activation='relu'))
        for i in range(num_layers-1):
                model.add(Dense(num_neurons[i], activation='relu'))
        model.add(Dense(2, activation='linear'))
        model.compile(
                            optimizer=keras.optimizers.RMSprop(learning_rate=l_rates, momentum=momentum, weight_decay=decays),
                            loss=['mean_squared_error'],
                            metrics=[mean_euclidean_error])
        return model

In [ ]:
def create_model_SGD(num_neurons, l_rates, decays, momentum, nesterov):
        num_layers = len(num_neurons)
        model = Sequential()
        model.add(Dense(num_neurons[0], input_shape=(9,), activation='relu'))
        for i in range(num_layers-1):
                model.add(Dense(num_neurons[i], activation='relu'))
        model.add(Dense(2, activation='linear'))
        model.compile(
                            optimizer=keras.optimizers.RMSprop(learning_rate=l_rates, momentum=momentum, weight_decay=decays, nesterov=nesterov),
                            loss=['mean_squared_error'],
                            metrics=[mean_euclidean_error])
        return model

# Model Selection

In [ ]:
callback = EarlyStopping(monitor='mean_euclidean_error', patience=100)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

## SGD

In [ ]:
model = KerasRegressor(build_fn=create_model_SGD, epochs=1000, batch_size=10, verbose=1)

In [ ]:
param_grid_SGD = {
    'num_neurons': [[20, 50, 50], [20, 50, 100], [50, 50, 100], [50, 100, 200], [100, 200, 200]],
    'l_rates' : [10**-i for i in range(3, 5)],
    'momentum' : [0.5, 0.7, 0.9],   
    'decays' : [0, 0.0001],
    'nesterov' : [True, False]
    }

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid_SGD, cv=cv, scoring=make_scorer(mean_euclidean_error_skit_friendly, greater_is_better=False))

In [ ]:
grid_result = grid.fit(X_train, y_train, callbacks=[callback])
best_SGD_model = grid.best_estimator_

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
y_pred_sgd = best_SGD_model.predict(X_train)
print('Best SGD model performance')
print('MSE: %.3f' %mean_squared_error(y_pred_sgd, y_test), 
      'MAE: %.3f' %mean_euclidean_error_skit_friendly(y_pred_sgd, y_test)
    )

## RMSprop

In [ ]:
model = KerasRegressor(build_fn=create_model_RMSprop, epochs=1000, batch_size=10, verbose=1)

In [ ]:
param_grid_RMSprop = {
    'num_neurons': [[20, 50, 50], [20, 50, 100], [50, 50, 100]],
    'l_rates' : [10**-i for i in range(3, 5)],
    'momentum' : [0, 0.1, 0.2],   
    'decays' : [0, 0.0001],
    }

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid_RMSprop, cv=cv, scoring=make_scorer(mean_euclidean_error_skit_friendly, greater_is_better=False))

In [ ]:
grid_result = grid.fit(X_train, y_train, callbacks=[callback])
best_RMSprop_model = grid.best_estimator_

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
y_pred_sgd = best_SGD_model.predict(X_train)
print('Best SGD model performance')
print('MSE: %.3f' %mean_squared_error(y_pred_sgd, y_test), 
      'MAE: %.3f' %mean_euclidean_error_skit_friendly(y_pred_sgd, y_test)
    )

# Plotting results

In [ ]:
history_SGD = best_SGD_model.fit(X_train, y_train, epochs=1000, batch_size=10, validation_data=(X_test, y_test), verbose=0)
history_RMSprop = best_RMSprop_model.fit(X_train, y_train, epochs=1000, batch_size=10, validation_data=(X_test, y_test), verbose=0)

In [ ]:
def plot_loss(history):
    loss_train = history.history['loss']
    loss_val = history.history['val_loss']

    plt.plot(loss_train, label='training', linestyle='--', color='#FA7070')
    plt.plot(loss_val, label='internal test', color='lightblue')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

def plot_error(history):
    err_train = history.history['mean_euclidean_error']
    err_val = history.history['val_mean_euclidean_error']

    plt.plot(err_train, label='training', linestyle='--', color='#FA7070')
    plt.plot(err_val, label='internal test', color='lightblue')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('MEE')
    plt.show()

In [ ]:
plot_loss(history_SGD)
plot_error(history_SGD)

In [ ]:
plot_loss(history_RMSprop)
plot_error(history_RMSprop)